<a href="https://colab.research.google.com/github/basantsomkuwar/ChhindwaraLocal/blob/main/Chhindwara_Local_Map_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required libraries
!pip install gspread google-auth folium

# Step 2: Authenticate with Google
from google.colab import auth
auth.authenticate_user()

import gspread
import google.auth
from google.auth import default

# Step 3: Create client to access Google Sheets
credentials, project = default()
gc = gspread.authorize(credentials)

# Step 4: Open your Google Sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/16gtG6it82N2xkrp2gy-15zoakKn9R_W_cfoXl5KKppc/edit?usp=sharing"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.sheet1

# Step 5: Get data and convert to DataFrame
import pandas as pd
data = worksheet.get_all_records()
df = pd.DataFrame(data)

print("Preview of your data:")
print(df.head())

# Step 6: Create map with hyperlinked markers
import folium

if not df.empty:
    # Create base map
    first_lat = df.iloc[0]['Latitude']
    first_lon = df.iloc[0]['Longitude']
    my_map = folium.Map(location=[first_lat, first_lon], zoom_start=10)

    # Add markers with hyperlinks
    for index, row in df.iterrows():
        # Customize these column names to match your sheet
        LINK_COLUMN = 'URL'  # Change this to your actual column name with links
        TITLE_COLUMN = 'Name'  # Change this to your title column

        # Create popup content with hyperlink
        popup_text = f"<div style='min-width:250px;'>"

        for col in df.columns:
            if col not in ['Latitude', 'Longitude']:
                value = row[col]
                # Make the link column clickable
                if col == LINK_COLUMN:
                    popup_text += (
                        f"<b>{col}:</b> "
                        f"<a href='{value}' target='_blank' style='color:blue;text-decoration:underline;'>"
                        f"Click here</a><br>"
                    )
                elif col == TITLE_COLUMN:
                    popup_text = f"<h4 style='margin-bottom:5px;'>{value}</h4>" + popup_text
                else:
                    popup_text += f"<b>{col}:</b> {value}<br>"

        popup_text += "</div>"

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color='green', icon='globe')
        ).add_to(my_map)

    print("\nDisplaying map with hyperlinked markers:")
    display(my_map)
else:
    print("No data found in the spreadsheet!")

